# Introduction to the JupyterLab and Jupyter Notebooks

This is a short introduction to two of the flagship tools created by [the Jupyter Community](https://jupyter.org).

> **⚠️Experimental!⚠️**: This is an experimental interface provided by the [JupyterLite project](https://jupyterlite.readthedocs.io/en/latest/). It embeds an entire JupyterLab interface, with many popular packages for scientific computing, in your browser. There may be minor differences in behavior between JupyterLite and the JupyterLab you install locally. You may also encounter some bugs or unexpected behavior. To report any issues, or to get involved with the JupyterLite project, see [the JupyterLite repository](https://github.com/jupyterlite/jupyterlite/issues?q=is%3Aissue+is%3Aopen+sort%3Aupdated-desc).

## JupyterLab 🧪

**JupyterLab** is a next-generation web-based user interface for Project Jupyter. It enables you to work with documents and activities such as Jupyter notebooks, text editors, terminals, and custom components in a flexible, integrated, and extensible manner. It is the interface that you're looking at right now.

**For an overview of the JupyterLab interface**, see the **JupyterLab Welcome Tour** on this page, by going to `Help -> Welcome Tour` and following the prompts.

> **See Also**: For a more in-depth tour of JupyterLab with a full environment that runs in the cloud, see [the JupyterLab introduction on Binder](https://mybinder.org/v2/gh/jupyterlab/jupyterlab-demo/HEAD?urlpath=lab/tree/demo).

## Jupyter Notebooks 📓

**Jupyter Notebooks** are a community standard for communicating and performing interactive computing. They are a document that blends computations, outputs, explanatory text, mathematics, images, and rich media representations of objects.

JupyterLab is one interface used to create and interact with Jupyter Notebooks.

**For an overview of Jupyter Notebooks**, see the **JupyterLab Welcome Tour** on this page, by going to `Help -> Notebook Tour` and following the prompts.

> **See Also**: For a more in-depth tour of Jupyter Notebooks and the Classic Jupyter Notebook interface, see [the Jupyter Notebook IPython tutorial on Binder](https://mybinder.org/v2/gh/ipython/ipython-in-depth/HEAD?urlpath=tree/binder/Index.ipynb).

## An example: visualizing data in the notebook ✨

Below is an example of a code cell. We'll visualize some simple data using two popular packages in Python. We'll use [NumPy](https://numpy.org/) to create some random data, and [Matplotlib](https://matplotlib.org) to visualize it.

Note how the code and the results of running the code are bundled together.

In [ ]:
# %% [markdown]
# # Telco Customer Churn Prediction
# ### Problem Statement & Objective
# **Business Problem**: Telecom companies lose 15-25% of customers annually to churn.  
# **ML Objective**: Build a classifier to predict churn with >0.60 F1-score.  
# **Success Metrics**:
# - Accuracy > 80%  
# - F1-score > 0.60  
# - ROC AUC > 0.75

# %% [markdown]
# ---
# ## 1. Dataset Loading
# **Source**: [IBM Telco Dataset](https://github.com/IBM/telco-customer-churn-on-icp4d)  
# **Format**: CSV (7,043 rows × 21 columns)  
# **License**: MIT

# %%
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, 
                           f1_score, 
                           confusion_matrix, 
                           classification_report,
                           roc_auc_score,
                           RocCurveDisplay)

# Load data
df = pd.read_csv("Telco-Customer-Churn.csv")
print(f"Data shape: {df.shape}")
display(df.head(3))


In [ ]:
# ## 2. Data Preprocessing
# **Key Steps**:
# 1. Handle missing values in TotalCharges
# 2. Convert categorical Churn to binary
# 3. Split features into numeric/categorical

# %%
# Clean TotalCharges
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges'])

# Convert target
df['Churn'] = df['Churn'].map({'Yes':1, 'No':0})

# Feature lists
numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
categorical_features = ['InternetService', 'Contract', 'PaymentMethod']

In [ ]:
# ## 3. Model Development
# **Pipeline Architecture**:
# 1. Numeric: Impute → Scale  
# 2. Categorical: Impute → OneHot  
# 3. Classifier: Logistic Regression

# %%
# Preprocessing
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)])

# Full pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(class_weight='balanced',
                                   max_iter=1000,
                                   solver='liblinear'))
])

# %% [markdown]

In [ ]:
# ## 4. Training & Evaluation
# **Strategy**: 80/20 train-test split with stratified sampling

# %%
# Split data
X = df[numeric_features + categorical_features]
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Train
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:,1]

# %% [markdown]
# ### Evaluation Metrics
# %%
# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

# Confusion Matrix
plt.figure(figsize=(6,4))
sns.heatmap(confusion_matrix(y_test, y_pred), 
            annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicted No', 'Predicted Yes'],
            yticklabels=['Actual No', 'Actual Yes'])
plt.title("Confusion Matrix")
plt.show()

# ROC Curve
RocCurveDisplay.from_predictions(y_test, y_proba)
plt.plot([0,1], [0,1], linestyle='--')
plt.title("ROC Curve")
plt.show()

In [ ]:
# ## 5. Insights & Deployment
# **Key Findings**:
# - Top 3 Predictive Features:
#   1. Contract type (Month-to-month)
#   2. Tenure length
#   3. Fiber optic internet

# %%
# Save model
joblib.dump(pipeline, 'churn_model.joblib')

# Feature Importance
encoder = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['encoder']
cat_features = encoder.get_feature_names_out(categorical_features)
all_features = numeric_features + list(cat_features)

pd.Series(pipeline.named_steps['classifier'].coef_[0],
          index=all_features).sort_values().plot(kind='barh')
plt.title("Logistic Regression Coefficients")
plt.show()

## Next steps 🏃

This is just a short introduction to JupyterLab and Jupyter Notebooks. See below for some more ways to interact with tools in the Jupyter ecosystem, and its community.

### Other notebooks in this demo

Here are some other notebooks in this demo. Each of the items below corresponds to a file or folder in the **file browser to the left**.

- [**`Lorenz.ipynb`**](Lorenz.ipynb) uses Python to demonstrate interactive visualizations and computations around the [Lorenz system](https://en.wikipedia.org/wiki/Lorenz_system). It shows off basic Python functionality, including more visualizations, data structures, and scientific computing libraries.
- [**`r.ipynb`**](r.ipynb) demonstrates the R programming language for statistical computing and data analysis.
- [**`cpp.ipynb`**](cpp.ipynb) demonstrates the C++ programming language for scientific computing and data analysis.
- [**`sqlite.ipynb`**](sqlite.ipynb) demonstrates how an in-browser sqlite kernel to run your own SQL commands from the notebook. It uses the [jupyterlite/xeus-sqlite-kernel](https://github.com/jupyterlite/xeus-sqlite-kernel).

### Other sources of information in Jupyter

- **More on using JupyterLab**: See [the JupyterLab documentation](https://jupyterlab.readthedocs.io/en/stable/) for more thorough information about how to install and use JupyterLab.
- **More interactive demos**: See [try.jupyter.org](https://try.jupyter.org) for more interactive demos with the Jupyter ecosystem.
- **Learn more about Jupyter**: See [the Jupyter community documentation](https://docs.jupyter.org) to learn more about the project, its community and tools, and how to get involved.
- **Join our discussions**: The [Jupyter Community Forum](https://discourse.jupyter.org) is a place where many in the Jupyter community ask questions, help one another, and discuss issues around interactive computing and our ecosystem.